In [245]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


### Assignment Owner: Tian Wang, Marylou Gabrié


#######################################
### Feature normalization
def feature_normalization(train, test):
    """Rescale the data so that each feature in the training set is in
    the interval [0,1], and apply the same transformations to the test
    set, using the statistics computed on the training set.

    Args:
        train - training set, a 2D numpy array of size(num_instances, num_features)
        test - test set, a 2D numpy array of size(num_instances, num_features)

    Returns:
        train_normalized - training set after normalization
        test_normalized - test set after normalization
    """
    for i in range(train.shape[1]):
        x_max = train[:,i].max()
        x_min = train[:,i].min()        
        # a is the rescale parameter, b is the shift parameter
        # a, b are trained from the training set
        # if the feature is constant, then do nothing to it
        if x_max == x_min:
            a = 1
            b = 0
        else:
            a = 1/(x_max-x_min)
            b = -x_min
        
        # apply the transformation on the train and test sets
        train[:,i] = a*(train[:,i]+b)
        test[:,i] = a*(test[:,i]+b)
        train_normalized = train
        test_normalized = test
    
    return train_normalized, test_normalized

#######################################
### The square loss function
    Given a set of X, y, theta, compute the average square loss for predicting y with X*theta.

    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        theta - the parameter vector, 1D array of size(num_features)

    Returns:
        loss - the average square loss, scalar
    """
    m = X.shape[0]
    norm = np.linalg.norm(X.dot(theta) - y)
    loss = norm**2/(m)
    
    return loss

#######################################
### The gradient of the square loss function
def compute_square_loss_gradient(X, y, theta):
    """
    Compute the gradient of the average square loss(as defined in compute_square_loss), at the point theta.

    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        theta - the parameter vector, 1D numpy array of size(num_features)

    Returns:
        grad - gradient vector, 1D numpy array of size(num_features)
    """
    m = X.shape[0]
    grad = 2*(X.T.dot(X).dot(theta) - X.T.dot(y))/m
    
    return grad


#######################################
### Gradient checker
#Getting the gradient calculation correct is often the trickiest part
#of any gradient-based optimization algorithm. Fortunately, it's very
#easy to check that the gradient calculation is correct using the
#definition of gradient.
#See http://ufldl.stanford.edu/wiki/index.php/Gradient_checking_and_advanced_optimization
def grad_checker(X, y, theta, epsilon=0.01, tolerance=1e-4):
    """Implement Gradient Checker
    Check that the function compute_square_loss_gradient returns the
    correct gradient for the given X, y, and theta.

    Let d be the number of features. Here we numerically estimate the
    gradient by approximating the directional derivative in each of
    the d coordinate directions:
(e_1 =(1,0,0,...,0), e_2 =(0,1,0,...,0), ..., e_d =(0,...,0,1))

    The approximation for the directional derivative of J at the point
    theta in the direction e_i is given by:
(J(theta + epsilon * e_i) - J(theta - epsilon * e_i)) /(2*epsilon).

    We then look at the Euclidean distance between the gradient
    computed using this approximation and the gradient computed by
    compute_square_loss_gradient(X, y, theta).  If the Euclidean
    distance exceeds tolerance, we say the gradient is incorrect.

    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        theta - the parameter vector, 1D numpy array of size(num_features)
        epsilon - the epsilon used in approximation
        tolerance - the tolerance error

    Return:
        A boolean value indicating whether the gradient is correct or not
    """
    true_gradient = compute_square_loss_gradient(X, y, theta) #The true gradient
    num_features = theta.shape[0]
    approx_grad = np.zeros(num_features) #Initialize the gradient we approximate
    #TODO
    for i in range(num_features):
        e_i = np.zeros(num_features)
        e_i[i] = 1
        approx_grad[i] = (compute_square_loss(X, y, theta+epsilon*e_i) - 
                         compute_square_loss(X, y, theta-epsilon*e_i)) / (2*epsilon)   
    # If the gradient is wrong, then return 0 
    if np.linalg.norm(true_gradient - approx_grad) > tolerance:
        indicator = 0
    # if the gradient is corret, then return 1
    else:
        indicator = 1
    
    return indicator


#######################################
### Generic gradient checker
def generic_gradient_checker(X, y, theta, objective_func, gradient_func, 
                             epsilon=0.01, tolerance=1e-4):
    """
    The functions takes objective_func and gradient_func as parameters. 
    And check whether gradient_func(X, y, theta) returned the true 
    gradient for objective_func(X, y, theta).
    Eg: In LSR, the objective_func = compute_square_loss, and gradient_func = compute_square_loss_gradient
    """
    true_gradient = gradient_func(X, y, theta)
    num_features = theta.shape[0]
    approx_grad = np.zeros(num_features)
    #TODO
    for i in range(num_features):
        e_i = np.zeros(num_features)
        e_i[i] = 1
        approx_grad[i] = (objective_func(X, y, theta+epsilon*e_i) - 
                         objective_func(X, y, theta-epsilon*e_i)) / (2*epsilon)   
    # If the gradient is wrong, then return 0 
    if np.linalg.norm(true_gradient - approx_grad) > tolerance:
        indicator = 0
    # if the gradient is corret, then return 1
    else:
        indicator = 1
    
    return indicator



#######################################
### Batch gradient descent
def batch_grad_descent(X, y, alpha=0.1, num_step=1000, grad_check=False):
    """
    In this question you will implement batch gradient descent to
    minimize the average square loss objective.

    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        alpha - step size in gradient descent
        num_step - number of steps to run
        grad_check - a boolean value indicating whether checking the gradient when updating

    Returns:
        theta_hist - the history of parameter vector, 2D numpy array of size(num_step+1, num_features)
                     for instance, theta in step 0 should be theta_hist[0], theta in step(num_step) is theta_hist[-1]
        loss_hist - the history of average square loss on the data, 1D numpy array,(num_step+1)
    """
    num_instances, num_features = X.shape[0], X.shape[1]
    theta_hist = np.zeros((num_step + 1, num_features))  #Initialize theta_hist
    loss_hist = np.zeros(num_step + 1)  #Initialize loss_hist
    theta = np.zeros(num_features)  #Initialize theta
    #TODO
    loss_hist[0] = compute_square_loss(X, y, theta)
    for i in range(num_step):
        grad = compute_square_loss_gradient(X, y, theta)
        if grad_check == True:
            check = grad_checker(X, y, theta, epsilon=0.01, tolerance=1e-4)
        theta = theta - alpha*grad       
        theta_hist[i+1,:] = theta
        loss = compute_square_loss(X, y, theta)
        loss_hist[i+1] = loss
    
    return loss_hist, theta_hist


#######################################
### The gradient of regularized batch gradient descent
def compute_regularized_square_loss_gradient(X, y, theta, lambda_reg):
    """
    Compute the gradient of L2-regularized average square loss function given X, y and theta

    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        theta - the parameter vector, 1D numpy array of size(num_features)
        lambda_reg - the regularization coefficient

    Returns:
        grad - gradient vector, 1D numpy array of size(num_features)
    """
    #TODO


#######################################
### Regularized batch gradient descent
def regularized_grad_descent(X, y, alpha=0.05, lambda_reg=10**-2, num_step=1000):
    """
    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        alpha - step size in gradient descent
        lambda_reg - the regularization coefficient
        num_step - number of steps to run
    
    Returns:
        theta_hist - the history of parameter vector, 2D numpy array of size(num_step+1, num_features)
                     for instance, theta in step 0 should be theta_hist[0], theta in step(num_step+1) is theta_hist[-1]
        loss hist - the history of average square loss function without the regularization term, 1D numpy array.
    """
    num_instances, num_features = X.shape[0], X.shape[1]
    theta = np.zeros(num_features) #Initialize theta
    theta_hist = np.zeros((num_step+1, num_features)) #Initialize theta_hist
    loss_hist = np.zeros(num_step+1) #Initialize loss_hist
    #TODO



#######################################
### Stochastic gradient descent
def stochastic_grad_descent(X, y, alpha=0.01, lambda_reg=10**-2, num_epoch=1000, eta0=False):
    """
    In this question you will implement stochastic gradient descent with regularization term

    Args:
        X - the feature vector, 2D numpy array of size(num_instances, num_features)
        y - the label vector, 1D numpy array of size(num_instances)
        alpha - string or float, step size in gradient descent
                NOTE: In SGD, it's not a good idea to use a fixed step size. Usually it's set to 1/sqrt(t) or 1/t
                if alpha is a float, then the step size in every step is the float.
                if alpha == "1/sqrt(t)", alpha = 1/sqrt(t).
                if alpha == "1/t", alpha = 1/t.
        lambda_reg - the regularization coefficient
        num_epoch - number of epochs to go through the whole training set

    Returns:
        theta_hist - the history of parameter vector, 3D numpy array of size(num_epoch, num_instances, num_features)
                     for instance, theta in epoch 0 should be theta_hist[0], theta in epoch(num_epoch) is theta_hist[-1]
        loss hist - the history of loss function vector, 2D numpy array of size(num_epoch, num_instances)
    """
    num_instances, num_features = X.shape[0], X.shape[1]
    theta = np.ones(num_features) #Initialize theta

    theta_hist = np.zeros((num_epoch, num_instances, num_features)) #Initialize theta_hist
    loss_hist = np.zeros((num_epoch, num_instances)) #Initialize loss_hist
    #TODO


def load_data():
    #Loading the dataset
    print('loading the dataset')

    df = pd.read_csv('ridge_regression_dataset.csv', delimiter=',')
    X = df.values[:,:-1]
    y = df.values[:,-1]

    print('Split into Train and Test')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, random_state=10)

    print("Scaling all to [0, 1]")
    X_train, X_test = feature_normalization(X_train, X_test)
    X_train = np.hstack((X_train, np.ones((X_train.shape[0], 1))))  # Add bias term
    X_test = np.hstack((X_test, np.ones((X_test.shape[0], 1))))
    return X_train, y_train, X_test, y_test

In [217]:
X_train, y_train, X_test, y_test = load_data()

loading the dataset
Split into Train and Test
Scaling all to [0, 1]


In [218]:
theta = np.ones(X_train.shape[1])

In [219]:
X_train.dot(theta).shape

(100,)

In [229]:
objective_func = compute_square_loss
gradient_func = compute_square_loss_gradient

generic_gradient_checker(X_train, y_train, theta, objective_func, gradient_func, 
                             epsilon=0.01, tolerance=1e-4)

1

In [251]:
X = X_train
y = y_train
alpha=0.1 
num_step=1000
grad_check=False


num_instances, num_features = X.shape[0], X.shape[1]
theta_hist = np.zeros((num_step + 1, num_features))  #Initialize theta_hist
loss_hist = np.zeros(num_step + 1)  #Initialize loss_hist
theta = np.zeros(num_features)  #Initialize theta
#TODO
loss_hist[0] = compute_square_loss(X, y, theta)
for i in range(num_step):
    grad = compute_square_loss_gradient(X, y, theta)
    if grad_check == True:
        check = grad_checker(X, y, theta, epsilon=0.01, tolerance=1e-4)
    theta = theta - alpha*grad       
    theta_hist[i+1,:] = theta
    loss = compute_square_loss(X, y, theta)
    loss_hist[i+1] = loss



<ipython-input-245-f601215e3793>:82: RuntimeWarning: overflow encountered in multiply
  grad = 2*(X.T.dot(X).dot(theta) - X.T.dot(y))/m
<ipython-input-251-789ba71ca009>:18: RuntimeWarning: invalid value encountered in subtract
  theta = theta - alpha*grad


In [279]:
X = X_train
y = y_train
alpha=0.01 
num_step=1000
grad_check=False
num_instances, num_features = X.shape[0], X.shape[1]
theta_hist = np.zeros((num_step + 1, num_features))  #Initialize theta_hist
loss_hist = np.zeros(num_step + 1)  #Initialize loss_hist
theta = np.zeros(num_features)  #Initialize theta


grad = compute_square_loss(X, y, theta)

In [286]:
grad = compute_square_loss_gradient(X, y, theta)
theta = theta - alpha*grad

In [287]:
theta

array([-0.01557088,  0.00047702,  0.01067574,  0.01394675, -0.00179555,
        0.00028419,  0.00265257,  0.00265257,  0.0121825 ,  0.02092759,
        0.02340332,  0.01091868, -0.00311404, -0.01082488,  0.02243974,
        0.02663839,  0.02163764,  0.00445784,  0.00148609,  0.00148609,
        0.00148609,  0.00476954,  0.00476954,  0.00476954,  0.00569285,
        0.00569285,  0.00569285,  0.00611209,  0.00611209,  0.00611209,
        0.00634201,  0.00634201,  0.00634201,  0.0087988 ,  0.0087988 ,
        0.0087988 ,  0.00967221,  0.00967221,  0.00967221,  0.00850654,
        0.00850654,  0.00850654,  0.00796286,  0.00796286,  0.00796286,
        0.00765993,  0.00765993,  0.00765993, -0.02027466])